In [1]:
import json
import os
from bs4 import BeautifulSoup
import re
import requests
from webob.compat import urlparse

In [2]:
class NaverFinanceNewsCrawler:
    URL_NAVER_FINANCE = "http://finance.naver.com"
    URL_NAVER_FINANCE_NEWS_QUERY = "http://finance.naver.com/news/news_search.nhn?q=%s&x=0&y=0" # params: query
    URL_NAVER_FINANCE_NEWS_CODE = "http://finance.naver.com/item/news_news.nhn?code=%s&page=%s" # params: (code, page)
    URL_NAVER_NEWS_FLASH = "http://finance.naver.com/news/news_list.nhn?mode=LSS2D&section_id=101&section_id2=258"
    URL_NAVER_STOCK_NOTICE = "http://finance.naver.com/item/news_notice.nhn?code=%s&page=%s" # params: (code, page)

    def __init__(self):
        pass

    def crawl(self, query=None, code=None, page=1):
        """

        :param query:
        :param code:
        :param page:
        :return:
        """
        if query:
            return self._crawl_by_query(query)
        elif code:
            return self._crawl_by_code(code, page=page)
        else:
            raise Exception("[Error] 'query' or 'code' should be entered.")

    def _crawl_by_query(self, query):
        """
        Crawl Naver Finance News
        :param query: string; search keywords
        :return: generator; [{title, summary, url, articleId, content, codes}, ...]
        """

        # Convert the query to euc-kr string
        q = ""
        for c in query.encode('euc-kr'):
            q += "%%%s" % format(c, 'x').capitalize()

        r_url = NaverFinanceNewsCrawler.URL_NAVER_FINANCE_NEWS_QUERY % (q)
        r = requests.get(r_url)

        soup = BeautifulSoup(r.text, "lxml")
        news = soup.find('div', class_='newsSchResult').find('dl', class_='newsList')
        news_title = news.find_all('dt', class_='articleSubject')
        news_summary = news.find_all('dd', class_='articleSummary')
        for title, summary in zip(news_title, news_summary):
            url = NaverFinanceNewsCrawler.URL_NAVER_FINANCE + title.a.get("href")
            res = {
                "title": title.a.text,
                "summary": summary.find(text=True).strip(' \t\n\r'),
                "url": url,
                "articleId": urlparse.parse_qs(urlparse.urlparse(url).query)["article_id"][0]
            }
            res.update(self._crawl_content(url))
            yield res

    def _crawl_by_code(self, code, page=10000):
        """
        Crawl Naver Stock News
        :param code: string; a stock code
        :return: generator;
        """

        r_url = NaverFinanceNewsCrawler.URL_NAVER_FINANCE_NEWS_CODE % (code, page)
        r = requests.get(r_url)

        soup = BeautifulSoup(r.text, "lxml")
        news_rows = soup.find('table', class_='type2').find_all('td', class_='title')

        for row in news_rows:
            yield {"title": row.a.text.strip(' \t\n\r'), "url": row.a.get('href')}

    def _crawl_content(self, url):
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "lxml")
        content = soup.find('div', id="content", class_='articleCont')
        codes = re.findall(r"\d{6}", content.text)
        return {"content": content.text.strip(' \t\n\r'), "codes": codes}

In [3]:
if __name__ == "__main__":
    crawler = NaverFinanceNewsCrawler()
    docs = crawler.crawl(query='셀바스AI')
    for i, d in enumerate(docs):
        print("{i}번째 문서".format(i=i+1), end=" " + "-" * 50)
        print("-" * 50)
        print("내용: {content}".format(content=d["content"]))
        print("문서에 포함된 종목 코드: {codes}".format(codes=d["codes"]))

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14516\1980643083.py:49: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  "summary": summary.find(text=True).strip(' \t\n\r'),


1번째 문서 ----------------------------------------------------------------------------------------------------
내용: 셀바스AI가 571억 규모의 유상증자에 성공했다.셀바스AI는 지난 21일부터 22일가지 기존 주주들을 대상으로 진행한 유상증자 청약률이 100.8%를 기록했다고 23일 밝혔다.발행 예정 주식수는 400만주로 총 571억원 규모이다. 이에 따라 26일부터 27일까지 진행 예정이었던 일반공모는 실시되지 않는다. 신주 상장 예정일은 7월 12일이다.셀바스AI는 이번 유상증자가 지난 4월 발표한 AI(인공지능)의료, 교육,  메타로빌리티(메타버스, 로봇, 모빌리티) 등 신사업 확대를 위해서라고 설명했다. 셀바스AI는 각 분야별 디지털 전환은 물론 경영 혁신을 지원한다는 계획이다.세부적으로 셀바스AI는 셀비 체크업, 셀비 메디보이스 등 제품을 통해 AI의료 사업을 본격 확대하고 있다. 올해부터는 군 이동형 원격의료도 본격화될 예정이다. 20225년부터 정부의 AI 디지털교과서 도입 등 인공지능이 교육 산업에도 본격 확대되는 것에 대응해 영어회화 솔루션 '토킹GPT'를 개발 하는 등 인공지능을 여러 분야로 확대시킬 수 있는 준비를 하고 있다.곽민철 셀바스AI 대표는 "당사의 성장계획을 신뢰하고 증자에 참여해주신 주주들에게 감사하다"며 "제시한 비전을 빠르게 실현해 차별화된 성장을 만들어 기업가치 상승과 주주가치 제고에 최선을 다할 것"이라고 말했다.머니투데이 관련뉴스해당 언론사에서 선정하며 언론사 페이지(아웃링크)로 이동해 볼 수 있습니다."예비 형수가 과거 원나잇 상대"…알려라 vs 참아라'이승기 갈등' 권진영, 마약 혐의 검찰행'케이티♥' 송중기, 애아빠 믿기지 않는 '동안외모'빽가 "제주 카페하다 인테리어 사업자 등록까지"엄정화, 팬사인회 도중 눈물 "콘서트 너무 하고 싶어"
문서에 포함된 종목 코드: []
2번째 문서 ---------------------------